In [8]:
##### Function to Hide Code found at: http://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [5]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, show


In [6]:
df = pd.read_csv("mock_student_data.csv")
columns = df.columns

In [22]:
df['Age'].hist()
plt.title("Age Distribution")
plt.savefig("Age")
plt.close()

In [23]:
df['GPA'].hist()
plt.title("GPA Distribution")
plt.savefig("GPA")
plt.close()

In [24]:
df['Days_missed'].hist()
plt.title("Days_missed Distribution")
plt.savefig("Days_missed")
plt.close()

In [ ]:
f['Days_missed'].hist()
plt.title("Days_missed Distribution")
plt.savefig("Days_missed")
plt.close()

# Age

In [199]:
age_summary = summary_table(df, 'Age')
age_summary

,mean,median,mode,standard_deviation,null_count
0,16.996109,17.0,15.0,1.458067,229


# GPA

In [200]:
gpa_summary = summary_table(df, 'GPA')
gpa_summary

,mean,median,mode,standard_deviation,null_count
0,2.988447,3.0,2.0,0.818249,221


# Days Missed

In [10]:
days_missed_summary = summary_table(df, 'Days_missed')
days_missed_summary

,mean,median,mode,standard_deviation,null_count
0,18.011139,18.0,6.0,9.629371,192
1,18.011139,18.0,14.0,9.629371,192
2,18.011139,18.0,31.0,9.629371,192


In [185]:
#infer gender for missing values with API 
fill_gender(df)


In [143]:
#### mean meth0d for missing data
df_mean = df.copy()
fill_mean(df_mean,"Age")
fill_mean(df_mean,"GPA")
fill_mean(df_mean,"Days_missed")
df_mean.to_csv("Student_Data_Filled_With_Mean.csv")


In [144]:
##### class  conditional mean method for missing data
df_conditional_mean = df.copy()
fill_conditional(df_conditional_mean,"Age")
fill_conditional(df_conditional_mean,"GPA")
fill_conditional(df_conditional_mean,"Days_missed")
df_conditional_mean.to_csv("Student_Data_Filled_With_Conditional_Mean.csv")


In [189]:
#### Possible Alternative solution
df_multiple_conditional_mean = df.copy()
fill_multiple_conditional(df_multiple_conditional_mean, "Age", check_gender= True)
fill_multiple_conditional(df_multiple_conditional_mean, "GPA", check_gender = True)
fill_multiple_conditional(df_multiple_conditional_mean, "Days_missed", check_gender = True)
df_conditional_mean.to_csv("Student_Data_Filled_With_Conditional_Mean_With_Gender.csv")



In [9]:
def summary_table(df,col):
    mode_list = []
    for mode in df[col].mode():
        mode_list.append(mode)
    summary = {'mean': df[col].mean(),
                'median':df[col].median(),
                'mode': mode_list,
                'standard_deviation': df[col].std(),
                'null_count': len(df[col]) - df[col].count()}
    summary_df = pd.DataFrame(summary, columns = ['mean', 'median', 'mode', 'standard_deviation', 'null_count'])
    return  summary_df

In [108]:
def fill_gender(df):
    missing = df.loc[df['Gender'].isnull(),'First_name']
    infer = []
    for name in missing:
        infer.append(infer_gender(name))
    df.loc[df['Gender'].isnull(),'Gender'] = infer

In [183]:
def infer_gender(name):
    url = "https://api.genderize.io/?name="
    request = url + name
    r = requests.get(request)
    gender = r.json()['gender']
    return gender.title()

In [102]:
def fill_mean(df, column_name):
    df.loc[df[column_name].isnull(), column_name] = df[column_name].mean()

In [137]:
def fill_conditional(df, col):
    GRAD = df['Graduated'] == "Yes"
    NOGRAD = df['Graduated'] == "No"
    COL_EMPTY = df[col].isnull()
    grad_col = df[GRAD][col].mean()
    nograd_col = df[NOGRAD][col].mean()

    df.loc[GRAD & COL_EMPTY, col] = grad_col
    df.loc[NOGRAD & COL_EMPTY, col] = nograd_col

In [188]:
def fill_multiple_conditional(df,col,check_gender = False):
    if not(check_gender):
        GRAD = df['Graduated'] == "Yes"
        NOGRAD = df['Graduated'] == "No"
        COL_EMPTY = df[col].isnull()
        grad_col = df[GRAD][col].mean()
        nograd_col = df[NOGRAD][col].mean()

        df.loc[GRAD & COL_EMPTY, col] = grad_col
        df.loc[NOGRAD & COL_EMPTY, col] = nograd_col
    if check_gender:
        GRAD = df['Graduated'] == "Yes"
        NOGRAD = df['Graduated'] == "No"
        COL_EMPTY = df[col].isnull()
        MALE = df['Gender'] == "Male"
        FEMALE = df['Gender'] == "Female"
        male_grad_col = df[MALE & GRAD][col].mean()
        female_grad_col = df[FEMALE & GRAD][col].mean()
        male_nograd_col = df[MALE & NOGRAD][col].mean()
        female_nograd_col = df[FEMALE & NOGRAD][col].mean()
        
        df.loc[GRAD & COL_EMPTY & MALE, col] = male_grad_col
        df.loc[NOGRAD & COL_EMPTY & MALE, col] = male_nograd_col
        df.loc[GRAD & COL_EMPTY & FEMALE, col] = female_grad_col
        df.loc[NOGRAD & COL_EMPTY & FEMALE, col] = female_nograd_col
        
        
        
        
